In [1]:
!pip install Korpora
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.2 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.5 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
import pandas as pd

train_1 = pd.read_csv('/content/drive/MyDrive/ITStudy/기술세미나3차/data/0725.csv',encoding='cp949',names=['id','document','label'])
train_2 = pd.read_csv('/content/drive/MyDrive/ITStudy/기술세미나3차/data/normal.csv',encoding='cp949',names=['id','document','label'])
train_3 = pd.read_csv('/content/drive/MyDrive/ITStudy/기술세미나3차/data/unnormal.csv',encoding='cp949',names=['id','document','label'])

train_2 = train_2.sample(10000)

test = pd.read_csv('/content/drive/MyDrive/ITStudy/기술세미나3차/data/0726.csv',encoding='cp949',names=['id','document','label'])
test1 = pd.read_csv('/content/drive/MyDrive/ITStudy/기술세미나3차/data/instagram.csv',encoding='cp949',names=['id','document','label'])

train = pd.concat([train_1,train_2,train_3])

In [4]:
train['length'] = train['document'].apply(lambda x: len(str(x)))
test['length'] = test['document'].apply(lambda x: len(str(x)))
test1['length'] = test1['document'].apply(lambda x: len(str(x)))

In [5]:
train.head(10)

,id,document,label,length
0,0.wav,현재 군 사건은 본인뿐만 아니라 전국에 걸쳐 180명도 연루가 된 광범위한 금룡제 ...,0,50
1,0.wav,저희가 원래 소환조사가 원칙이지만 180명 연루가 돼 있다 보니 저희가 다 일일이 ...,0,164
2,0.wav,알겠습니다. 진술받았고요. 마지막 진술을 받고 담당 검사님이 이관을 시켜드릴 거예요...,0,89
3,0.wav,이상 말씀드리겠습니다. 제3자의 육성 또는 기타 소문 들릴 시 녹취파일스 김주라는 ...,0,118
4,0.wav,가 보니 위치하신 곳이 직장이신 거예요 아니면 자택이신 거예요? 직장이시면 혹시 메...,0,86
5,0.wav,욕실 진행을 할 거예요. 본인 육성만 들러야 되는데요. 제3차 육성이나 기타 선으로...,0,95
6,0.wav,"다 대면이고, 본인과 연루된 명의도용 사건이 접수가 돼서 몇 가지 흑인차 연락을 드...",0,51
7,0.wav,혹시 본인 김천우라고 아시는 분이신가요? 처음 들어보신 분이에요. 저희 쪽에서는 사...,0,77
8,0.wav,"알겠습니다. 우리은행에는 입출금하나 청약하나 따로 와인성장, 정기예금 전권 부분은 ...",0,78
9,0.wav,입출금 한 게 농약 부분에도 입출금 한 개만 있으신 것 같고 적금이나 다른 정기 예...,0,154


In [6]:
 train.loc[train['length']<5]['document'].unique()

array(['후', '어', '그', '쉬고', nan, '되죠.', '많아요.', '요,', '..', '.', '...',
       '한 분이', '그렇지', '어색해', '그렇구나', '보여줘?', '짱이군', '재밌겠당', '아니야', '실화냐',
       '맞아!', '나야나!', '그러게', '아 맞아', '아 웃겨', '맞아요', '아 맞다', '모르지', '그래?',
       '고맙겠다', '그러게요', '멋있음', '웃기네', '상체쪽?', '언제?', '그럴듯', '그래 뭐', '좋겠다',
       '착각도.', '웅 좋아', '그러게,', '볼링?', '수학?', '맞습니다', '왜죠?', '사표라니', '좋아~',
       'RGRG', '그럴껄?', '부럽네', '알았어', '웅 맞아', '알바?', '오 좋다', '죽겠어', '마자요',
       '재밌겠다', '참나.', '누가?', '헐 대박', '맛있더라', '그렇죠', '아하…', '모르겠네', '아니냐?',
       '아니아니', '흐엉 ㅜ', '아쉽네요', '수중?', '이상해', '개부럽다', '아니요', '강화도?', '복불복',
       '심하네', '이해해', '하긴.', '그치!', '힘드네요', '난 별로', '엉 키키', '진짜?', '그렇조',
       '응 키키', '어 키키', '그렇죠?', '줄여야지', '옹 키키', '아 그치', '봐야겠다', '부러워',
       '쌤은요?', '이제?', '아 예', '네', '예', '아', '아 네', '예 예', '네 네', '그러면',
       '네네.', '예?', '네.', '네 네.', '네네네.', '예.', '아...', '음...', '예,',
       '네 .', '예 .', '네?', '어...', '하...', '예 예.', '아 네.', '응.', '여보세요',
       '아 예.', '아~', '음 네네', '음', '그쵸', '2주요?', '

In [7]:
train = train.loc[train['length'] > 5]

test1.dropna(inplace=True)
test = test.loc[test['length'] > 5]
test1 = test1.loc[test1['length']>5]

In [8]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [9]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader


class TokenDataset(Dataset):

    def __init__(self, dataframe, tokenizer_pretrained):
        self.data = dataframe
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        token_type_ids = torch.zeros_like(attention_mask)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)


In [10]:
tokenizer_pretrained = CHECKPOINT_NAME

train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [11]:
inputs, labels = next(iter(train_loader))

inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

tensor([0, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [12]:
from transformers import BertConfig

config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [13]:
from transformers import BertModel

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [14]:
output = model_bert(**inputs)

In [15]:
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

torch.Size([8, 512, 768])
tensor([[ 0.0083, -0.2762,  1.1329,  ..., -0.0041,  0.3225,  0.0305],
        [-0.1919, -0.5448,  0.7177,  ...,  0.2628, -0.0701,  0.1964],
        [ 0.0590, -0.3128,  0.1369,  ..., -0.0834, -0.1685,  0.7568],
        ...,
        [ 0.1220,  0.1314,  0.5528,  ...,  0.1604, -0.1225,  0.5807],
        [ 0.1481, -0.1275,  1.4186,  ..., -0.2302, -0.5491,  0.2361],
        [-0.6074, -0.7044, -0.5326,  ...,  0.1125, -0.4287, -0.1426]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [16]:
pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

torch.Size([8, 768])
tensor([[-0.9271,  0.1389, -0.9673,  ...,  0.7062,  0.0632,  0.4057],
        [-0.9038, -0.1098, -0.9573,  ...,  0.1306,  0.3572, -0.3327],
        [ 0.8459,  0.4281, -0.9999,  ..., -0.3978,  0.7505,  0.2600],
        ...,
        [-0.6718,  0.2350, -0.6287,  ..., -0.2506, -0.0760,  0.8128],
        [-0.0918, -0.2045, -0.9995,  ...,  0.9288,  0.7547,  0.4454],
        [ 0.5728,  0.1140, -0.9987,  ...,  0.6613, -0.0200, -0.8703]],
       device='cuda:0', grad_fn=<TanhBackward0>)


In [17]:
fc = nn.Linear(768, 2)
fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

torch.Size([8, 2])
tensor([0, 1, 0, 1, 0, 0, 0, 0], device='cuda:0')


In [18]:
class BertFinetuningModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        super(BertFinetuningModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_pretrained)
        self.dr = nn.Dropout(p=dropout_rate)
        self.fc = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = output['last_hidden_state']
        x = self.dr(last_hidden_state[:, 0, :])
        x = self.fc(x)
        return x

In [19]:
# CustomBertModel 생성
bert = BertFinetuningModel(CHECKPOINT_NAME)
bert.to(device)

BertFinetuningModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [21]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    model.train()

    running_loss = 0
    corr = 0
    counts = 0

    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)

    for idx, (inputs, labels) in enumerate(prograss_bar):
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        output = model(**inputs)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()

        _, pred = output.max(dim=1)

        corr += pred.eq(labels).sum().item()
        counts += len(labels)

        running_loss += loss.item() * labels.size(0)

        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")

    acc = corr / len(data_loader.dataset)

    return running_loss / len(data_loader.dataset), acc

In [22]:
def model_evaluate(model, data_loader, loss_fn, device):
    model.eval()

    with torch.no_grad():
        corr = 0
        running_loss = 0

        for inputs, labels in data_loader:
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            output = model(**inputs)
            _, pred = output.max(dim=1)
            corr += torch.sum(pred.eq(labels)).item()
            running_loss += loss_fn(output, labels).item() * labels.size(0)

        acc = corr / len(data_loader.dataset)

        return running_loss / len(data_loader.dataset), acc

In [23]:
num_epochs = 1

model_name = 'bert-kor-base'

min_loss = np.inf

for epoch in range(num_epochs):
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)

    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')

    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 0.63737, training accuracy: 0.97157: 100%|██████████| 1887/1887 [07:32<00:00,  4.17batch/s]


[INFO] val_loss has been improved from inf to 0.00025. Saving Model!
epoch 01, loss: 0.07969, acc: 0.97157, val_loss: 0.00025, val_accuracy: 1.00000


In [24]:
bert.load_state_dict(torch.load(f'{model_name}.pth'))

<All keys matched successfully>

In [25]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels

    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')
        return self.labels[output]

In [26]:
tokenizer = BertTokenizerFast.from_pretrained(CHECKPOINT_NAME)

labels = {
    0: '보이스피싱 입니다.',
    1: '일반전화 입니다.'
}

predictor = CustomPredictor(bert, tokenizer, labels)

In [27]:
def predict_sentence(predictor):
    input_sentence = input('문장을 입력해 주세요: ')
    predictor.predict(input_sentence)

In [28]:
# 일상 대화 예시
predict_sentence(predictor)

문장을 입력해 주세요: 점심 메뉴 정하신 분? 누룽지 끓여 먹을까 고민 나 아침 든든하게 먹으니 힘 난다 아 나 점심 뭐 먹지 점심은 사모님이 주심 후후 나 점심 내장국밥 픽
[일반전화 입니다.]
확률은: 99.928% 입니다.


In [29]:
# 보이스피싱 통화 예시 1
predict_sentence(predictor)

문장을 입력해 주세요: 예 (삐-) 고객님 맞으시죠? 아 네 그 심사 결과 안내 차 연락 드렸는데 우선 대출 면에서는 지금 (삐-)께서 이렇게 대출을 받아 보셨지만 뭐 최근까지 연체 이력 한 번도 없으신 점과 (삐-) 어쨌든 5개 금융사 카드 이용 중이시잖아요? 아 이 부분도 연체 없이 또 우량하게 사용하시고 그 신용 거래를 오래 해주신 점은 저희 국민 은행 측에서 좀 굉장히 긍정적으로 평가를 하세요 근데 반면에 아무래도 그 대출 면에서 (삐-) 이용하시는데 보험사는 2금융권이세요. 그래서 전산에서는 그 1금융권 거래 실적이 좀 미비하시다라고 하는데요. 그 다행히도 그 상환 능력 평가 미달로 부결이 나신 거라서, 이 상환 능력 평가 확인되시면 좀 더 좋은 조건으로 발급이 가능하시다 확인이 되시니까
[보이스피싱 입니다.]
확률은: 99.987% 입니다.


In [30]:
# 보이스피싱 통화 예시 2
predict_sentence(predictor)

문장을 입력해 주세요: 예 현대 캐피탈 담당자... 아 네네네네. 예 예 고객님. 일단은 승인 결과 나오셔서 연락 좀 드렸구요. 통화는 좀 괜찮으세요? 예 지금 금액 건은 제가 최초에 내어드린 가지불 결과 좀 인지하고 계신가요? 예 2400만 원에 7.5% 말씀 드렸잖아요. 지금 금액 건은 기존보다 동일하게 내려오셨는데요 아 그런데 고객님께서는 현재 정상 승인권은 아니시고, 좀 죄송한 말씀이지만 지금 조건부 승인권으로 나와계시거든요.
[보이스피싱 입니다.]
확률은: 99.990% 입니다.
